In [1]:
import pandas as pd
from tqdm import tqdm
tqdm.pandas(desc='process')

In [2]:
cust_info = (pd.read_csv('dataset/public_train_x_custinfo_full_hashed.csv')
             .rename(columns=lambda x: x.lower()))
alert_key = pd.read_csv('dataset/train_x_alert_date.csv')
sar_flg = pd.read_csv('dataset/train_y_answer.csv')

In [3]:
case = (alert_key
        .merge(sar_flg, on='alert_key')
        .merge(cust_info, on='alert_key'))

### case base
#### 資料選取:
* drop date < 28 
* drop duplicates, keep last

In [4]:
case_info = (case
             .assign(
                 case_cnt=case.groupby(['cust_id'])['date'].rank(),
                 case_dt_last_diff=case.groupby(['cust_id'])['date'].diff().fillna(0),
                 case_dt_min=case.groupby('cust_id')['date'].transform('min'),
                 case_dt_min_diff=lambda df: df['date'] - df['case_dt_min'])
             .query("date > 27")
             .sort_values(['cust_id','date', 'sar_flag'],)
             .drop_duplicates(subset=['cust_id'], keep='last'))
             

In [5]:
case_info.head()

,alert_key,date,sar_flag,cust_id,risk_rank,occupation_code,total_asset,age,case_cnt,case_dt_last_diff,case_dt_min,case_dt_min_diff
4353,195913,53,0,0015cc01f553512007705edf855baa16f1b57a16fb107e...,1,17.0,32758.0,3,1.0,0.0,53,0
13458,271556,221,0,002d6bc6382e759c9a5b6bb2c8dda9add2f38cd87e10ca...,3,4.0,629787.0,3,3.0,3.0,217,4
16322,294760,263,0,002f13f534d0bd9fcc1848db3ff5c7b1179d117f8de805...,3,15.0,980597.0,4,4.0,3.0,153,110
5740,206471,78,0,0031e83ddef78e57d17ac5dce088627b7745be8ff6cffc...,2,13.0,406.0,5,6.0,1.0,70,8
21678,334035,330,0,0032adaf4a8731b4d6a3fdcba32f9ebdcd336490eda155...,3,12.0,4872215.0,2,43.0,2.0,106,224


In [6]:
case_info.sar_flag.value_counts()

0    5324
1     215
Name: sar_flag, dtype: int64

將過去28天分成4個週期<br>
先計算每個週期in/out/count/...<br>
再比較週期之間的差異<br>

In [7]:
def add_cycle(dt, dt_end):
    bins = [dt_end - i * 7 for i in range(4)]
    cycle = len([b for b in bins if b >= dt])
    return cycle


def create_stat_features(df):
    df = (df
          .groupby(['cycle', 'debit_credit'])['tx_amt'].describe()
          .stack().reset_index()
          .rename(columns={0: 'val'})
          .assign(features=lambda df: df['cycle'].astype(str) + df['debit_credit'] + df['level_2'])
          [['features', 'val']].set_index('features').T)
    return df


def create_diff_features(df):
    pass


def create_txn_features(df):
    pass


def create_features(cust_id, dt):
    try:
        tmp_df = (dp
                  .query("cust_id == @cust_id and tx_date < @dt and tx_date >= @dt - 28")
                  .assign(
                      cycle=lambda df: df.apply(lambda x: add_cycle(x['tx_date'], dt), axis=1))
                  .pipe(create_stat_features)
                  .assign(cust_id=cust_id))
    except Exception as e:
        tmp_df = pd.DataFrame()
        dp_tmp = dp.query("cust_id == @cust_id and tx_date < @dt and tx_date >= @dt - 28")
        if dp_tmp.empty:
            dp_tot_cnt = len(dp.query("cust_id == @cust_id"))
            alert_case = case_info.query("cust_id == @cust_id")
            if alert_case['sar_flag'].values[0] == 1:
                print(f"""{cust_id}|{dt}|{e}|dp tot cnt {dp_tot_cnt}|alert cnt {alert_case['case_cnt'].values[0]}|sar {alert_case['sar_flag'].values[0]}""")
        else:
            print(f"{cust_id}|{dt}|{e}")
    return tmp_df

In [8]:
dp = pd.read_csv('dataset/public_train_x_dp_full_hashed.csv')

In [9]:
res = case_info.progress_apply(lambda x: create_features(x['cust_id'], x['date']), axis=1)
res = (pd
       .concat(res.values)
       .set_index('cust_id').reset_index())

process:  22%|██████████████▉                                                      | 1197/5539 [01:27<05:54, 12.25it/s]

36c96f886054b13855e0531cfc53562de5acdada08fb75c24e6d957c0d8af1a9|35|'cycle'|dp tot cnt 79|alert cnt 1.0|sar 1


process:  87%|████████████████████████████████████████████████████████████▎        | 4843/5539 [05:51<00:49, 14.02it/s]

deea7839000d9f8c1e3fed53e0a1141266fb8d7149a05008bc4de475b8785c64|81|'cycle'|dp tot cnt 0|alert cnt 17.0|sar 1


process: 100%|█████████████████████████████████████████████████████████████████████| 5539/5539 [06:43<00:00, 13.72it/s]


In [10]:
res.shape

(4736, 65)

In [11]:
res.head()

features,cust_id,1CRcount,1CRmean,1CRmin,1CR25%,1CR50%,1CR75%,1CRmax,1DBcount,1DBmean,...,4CR75%,4CRmax,4DBcount,4DBmean,4DBstd,4DBmin,4DB25%,4DB50%,4DB75%,4DBmax
0,0015cc01f553512007705edf855baa16f1b57a16fb107e...,1.0,516.000000,516.0,516.00,516.0,516.0,516.0,1.0,826.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,002d6bc6382e759c9a5b6bb2c8dda9add2f38cd87e10ca...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,002f13f534d0bd9fcc1848db3ff5c7b1179d117f8de805...,6.0,14150.500000,521.0,2344.75,13544.5,23959.0,31251.0,2.0,216050.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0031e83ddef78e57d17ac5dce088627b7745be8ff6cffc...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,31312.0,...,31312.00,521874.0,5.0,10045.6,9101.014136,1044.0,1804.00,10453.0,13965.00,22962.0
4,0032adaf4a8731b4d6a3fdcba32f9ebdcd336490eda155...,31.0,236716.580645,30.0,500.00,2537.0,37433.0,2001768.0,5.0,12460.8,...,37683.75,50044.0,2.0,17488.5,9096.928740,11056.0,14272.25,17488.5,20704.75,23921.0
